# Assignment

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from pdb import set_trace
import math

In [2]:
# Import 

import numpy as np
import matplotlib.pyplot as plt
from degree_freedom_queen import *
from degree_freedom_king1 import *
from degree_freedom_king2 import *
from generate_game import *
from Chess_env import *


size_board = 4

## The Environment

You can find the environment in the file Chess_env, which contains the class Chess_env. To define an object, you need to provide the board size considered as input. In our example, size_board=4. 
Chess_env is composed by the following methods:

1. Initialise_game. The method initialises an episode by placing the three pieces considered (Agent's king and queen, enemy's king) in the chess board. The outputs of the method are described below in order.

     S $\;$ A matrix representing the board locations filled with 4 numbers: 0, no piece in that position; 1, location of the 
     agent's king; 2 location of the queen; 3 location of the enemy king.
     
     X $\;$ The features, that is the input to the neural network. See the assignment for more information regarding the            definition of the features adopted. To personalise this, go into the Features method of the class Chess_env() and change        accordingly.
     
     allowed_a $\;$ The allowed actions that the agent can make. The agent is moving a king, with a total number of 8                possible actions, and a queen, with a total number of $(board_{size}-1)\times 8$ actions. The total number of possible actions correspond      to the sum of the two, but not all actions are allowed in a given position (movements to locations outside the borders or      against chess rules). Thus, the variable allowed_a is a vector that is one (zero) for an action that the agent can (can't)      make. Be careful, apply the policy considered on the actions that are allowed only.
     

2. OneStep. The method performs a one step update of the system. Given as input the action selected by the agent, it updates the chess board by performing that action and the response of the enemy king (which is a random allowed action in the settings considered). The first three outputs are the same as for the Initialise_game method, but the variables are computed for the position reached after the update of the system. The fourth and fifth outputs are:

     R $\;$ The reward. To change this, look at the OneStep method of the class where the rewards are set.
     
     Done $\;$ A variable that is 1 if the episode has ended (checkmate or draw).
     
     
3. Features. Given the chessboard position, the method computes the features.

This information and a quick analysis of the class should be all you need to get going. The other functions that the class exploits are uncommented and constitute an example on how not to write a python code. You can take a look at them if you want, but it is not necessary.






In [3]:
## INITIALISE THE ENVIRONMENT

env=Chess_Env(size_board)

In [4]:
## PRINT 5 STEPS OF AN EPISODE CONSIDERING A RANDOM AGENT

S,X,allowed_a=env.Initialise_game()                       # INTIALISE GAME

print(S)                                                  # PRINT CHESS BOARD (SEE THE DESCRIPTION ABOVE)

print('check? ',env.check)                                # PRINT VARIABLE THAT TELLS IF ENEMY KING IS IN CHECK (1) OR NOT (0)
print('dofk2 ',np.sum(env.dfk2_constrain).astype(int))    # PRINT THE NUMBER OF LOCATIONS THAT THE ENEMY KING CAN MOVE TO


for i in range(5):
    
    a,_=np.where(allowed_a==1)                  # FIND WHAT THE ALLOWED ACTIONS ARE
    a_agent=np.random.permutation(a)[0]         # MAKE A RANDOM ACTION

#     set_trace()
    S,X,allowed_a,R,Done=env.OneStep(a_agent)   # UPDATE THE ENVIRONMENT
    
    
    ## PRINT CHESS BOARD AND VARIABLES
    print('')
    print(S)
    print(R,'', Done)
    print('check? ',env.check)
    print('dofk2 ',np.sum(env.dfk2_constrain).astype(int))
    
    
    # TERMINATE THE EPISODE IF Done=True (DRAW OR CHECKMATE)
    if Done:
        break
        


[[0 0 3 0]
 [0 0 0 0]
 [0 0 1 2]
 [0 0 0 0]]
check?  0
dofk2  0

[[0 0 3 0]
 [0 0 0 0]
 [0 1 0 2]
 [0 0 0 0]]
0.0  1
check?  0
dofk2  0


In [5]:
# PERFORM N_episodes=1000 EPISODES MAKING RANDOM ACTIONS AND COMPUTE THE AVERAGE REWARD AND NUMBER OF MOVES 

S,X,allowed_a=env.Initialise_game()
N_episodes=1000

# VARIABLES WHERE TO SAVE THE FINAL REWARD IN AN EPISODE AND THE NUMBER OF MOVES 
R_save_random = np.zeros([N_episodes, 1])
N_moves_save_random = np.zeros([N_episodes, 1])

for n in range(N_episodes):
    
    S,X,allowed_a=env.Initialise_game()     # INITIALISE GAME
    Done=0                                  # SET Done=0 AT THE BEGINNING
    i=1                                     # COUNTER FOR THE NUMBER OF ACTIONS (MOVES) IN AN EPISODE
    
    # UNTIL THE EPISODE IS NOT OVER...(Done=0)
    while Done==0:
        
        # SAME AS THE CELL BEFORE, BUT SAVING THE RESULTS WHEN THE EPISODE TERMINATES 
        
        a,_=np.where(allowed_a==1)
        a_agent=np.random.permutation(a)[0]

        S,X,allowed_a,R,Done=env.OneStep(a_agent)
        
        
        if Done:
            
            R_save_random[n]=np.copy(R)
            N_moves_save_random[n]=np.copy(i)

            break

        i=i+1                               # UPDATE THE COUNTER



# AS YOU SEE, THE PERFORMANCE OF A RANDOM AGENT ARE NOT GREAT, SINCE THE MAJORITY OF THE POSITIONS END WITH A DRAW 
# (THE ENEMY KING IS NOT IN CHECK AND CAN'T MOVE)

print('Random_Agent, Average reward:',np.mean(R_save_random),'Number of steps: ',np.mean(N_moves_save_random))



Random_Agent, Average reward: 0.217 Number of steps:  6.828


## Defining a simple NN 

In [32]:
class NeuralNetwork: 
    
    def __init__(self, Xavier_init= True, layer_type = 'ReLU',number_of_layers = 1, number_of_neurons_per_layer = [200], output_size = 1, input_size = 58):
        
        self.Ws = []
        self.bs = []
        self.layer_type = layer_type
        
        if Xavier_init:
            W1 = np.random.randn(number_of_neurons_per_layer[0], input_size) * np.sqrt(1 / (input_size))  # first layer 
            self.Ws.append(W1)
            for i in range(1, number_of_layers):
            
                W = np.random.randn(number_of_neurons_per_layer[i], number_of_neurons_per_layer[i-1]) * np.sqrt(1 / (number_of_neurons_per_layer[i-1]))
                self.Ws.append(W)
                
            W_out = np.random.randn(output_size, number_of_neurons_per_layer[-1]) * np.sqrt(1 / (number_of_neurons_per_layer[-1]))
            self.Ws.append(W_out)
            
            
        else: 
            print('Not Implemented :=(')
            

        for i in range(number_of_layers): 
            b = np.zeros((number_of_neurons_per_layer[i],))
            self.bs.append(b)
        b_out = np.zeros((output_size,))
        self.bs.append(b_out)
        
        
    def forward(self, x0): 
        
        layer_out = x0
        for i in range(len(self.Ws) - 1):
            layer_out_pre_activation = np.dot(self.Ws[i], layer_out) + self.bs[i]   # calculate Wx + b
            layer_out = 1/(1+np.exp(-layer_out_pre_activation))                     # apply sigmoid funciton 
            
        return layer_out_pre_activation  # because the last layer should have no sigmoid! 
    
    def _forward_batch(self, x0s): 
        outputs = [] 
        intermediate_outputs_list = [] 
        
        for x0 in x0s:
            output, intermediate_outputs = self._forward(x0)
            
            outputs.append(output)
            intermediate_outputs_list.append(intermediate_outputs)
        
        return outputs, intermediate_outputs_list
            
            
    
    
    def _forward(self, x0): 
        
        intermediate_outputs = [] 
        layer_out = x0
        intermediate_outputs.append(layer_out)
        
        if self.layer_type == 'ReLU': 
            for i in range(len(self.Ws) - 1):
                layer_out_pre_activation = np.dot(self.Ws[i], layer_out) + self.bs[i]   # calculate Wx + b
                layer_out = layer_out_pre_activation.copy()
                layer_out[layer_out< 0] = 0    # for ReLU: make all positions <0 equal to 0
                intermediate_outputs.append(layer_out)
                
            layer_out = np.dot(self.Ws[-1], layer_out) + self.bs[-1]
            intermediate_outputs.append(layer_out)
            
            return layer_out, intermediate_outputs
                
                
            
        else: 
            print('entering sigmoid part of forward')

            for i in range(len(self.Ws) - 1):
                layer_out_pre_activation = np.dot(self.Ws[i], layer_out) + self.bs[i]   # calculate Wx + b
                layer_out = 1/(1+np.exp(-layer_out_pre_activation))                     # apply sigmoid funciton
                intermediate_outputs.append(layer_out)

            layer_out = np.dot(self.Ws[-1], layer_out) + self.bs[-1]
            intermediate_outputs.append(layer_out)
            
            return layer_out, intermediate_outputs
    
    
    def backward(self, x0s, outputs, desired_outputs, intermediate_outputs_list): 
        
        
        dWs = [np.zeros(shape = self.Ws[i].shape) for i in range(len(self.Ws))]
        dbiases = [np.zeros(shape = self.bs[i].shape) for i in range(len(self.bs))]  
        
        
        
        for j in range(len(x0s)): 
            
            output = outputs[j]
            desired_output = desired_outputs[j]
            intermediate_outputs = intermediate_outputs_list[j]
            x0 = x0s[j]
            
            
            error_signal = desired_output - output

            if self.layer_type == 'ReLU': 
                
                for i in reversed(range(len(self.Ws))):
                    if i == len(self.Ws) - 1:
                        delta = error_signal                  # error signal -> L,  delta x input at layer-> Derivative at that layer    :=>    dW = derivative * lr 
                        
                    else: 
                        intermediate_outputs_copy = intermediate_outputs[i+1].copy()
                        intermediate_outputs_copy[intermediate_outputs_copy < 0] = 0 
                        intermediate_outputs_copy[intermediate_outputs_copy > 0 ] = 1
                        
                        delta = intermediate_outputs_copy * error_signal
                    
#                     set_trace()
                    
                    dWs[i] = dWs[i] +  np.outer(delta, intermediate_outputs[i])
                    dbiases[i] = dbiases[i] + delta
                    
                    error_signal = np.dot(self.Ws[i].T, delta)     
            
            else: 
                print('backward entering sigmoid part!')
            
                for i in reversed(range(len(self.Ws))):

                    if i == len(self.Ws) - 1:
    #                     print('last layer!')
                        delta = error_signal
                    else:
                        delta = intermediate_outputs[i+1]*(1-intermediate_outputs[i+1]) * error_signal

                    dWs[i] = dWs[i] +  np.outer(delta, intermediate_outputs[i])
                    dbiases[i] = dbiases[i] + delta

                    error_signal = np.dot(self.Ws[i].T, delta)
                
        return dWs, dbiases
    
    def update(self, dWs, dbiases, batch_size, lr): 
        
        for i in range(len(self.Ws)): 
            self.Ws[i] = self.Ws[i] + (dWs[i] * lr) / batch_size 
            self.bs[i] = self.bs[i] + (dbiases[i] * lr) / batch_size
            
            
        
    def train(self, x_train, y_train, lr, epochs, batch_size, print_frequency = 10 ,shuffle_indexes = True):
        
        number_of_batches = int(math.floor(len(x_train)/batch_size))  
        
        print('---Before training---')
        for j in range(len(self.Ws)):
            print(f'Sum of Ws for layer {j}: {np.sum(np.abs(self.Ws[j]))} and sum of bs: {np.sum(np.abs(self.bs[j]))}')
        
        for i in range(epochs): 
            
            if shuffle_indexes: 
                shuffled_idxs = np.random.permutation(X_train.shape[0])
            else: 
                shuffled_idxs = np.array([i for i in range(len(X_train))])
                
                
            for batch in range(0, number_of_batches): 
                
                idxs_current_batch = shuffled_idxs[batch * batch_size : (batch + 1) * batch_size]

                X_batch = X_train[idxs_current_batch]
                y_batch = y_train[idxs_current_batch]


                outputs, intermediate_outputs_list =  self._forward_batch(x0s= X_batch )
                dWs, dbiases = self.backward(x0s = X_batch, outputs = outputs, desired_outputs = y_batch,intermediate_outputs_list = intermediate_outputs_list)
                

                self.update(dWs, dbiases, batch_size, lr)
                
            if (i % print_frequency == 0): 
                print(f'Epoch: {i}')
                for j in range(len(self.Ws)): 
                    print(f'sum of abs dWs for layer {j}: {np.sum(np.abs(dWs[j]))} and sum of dbs: {np.sum(np.abs(dbiases[j]))}')
                    print(f'sum of Ws for layer {j}: {np.sum(np.abs(self.Ws[j]))} and sum of bs: {np.sum(np.abs(self.bs[j]))}')


                y_pred, _ = self._forward_batch(x0s = X_train)
                print(f'MSE on the training set: { np.square((np.array(y_pred) - np.array(y_train))).sum() / np.array(y_pred).shape[0] }')
                    
  

In [17]:
X_train = np.array([[i] for i in range(100)])

y_train = 2 * X_train

In [18]:
our_nn._forward(X_train[15])

(array([74.39682141]),
 [array([15]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
  array([74.39682141])])

In [38]:
our_nn = NeuralNetwork(number_of_layers= 3, number_of_neurons_per_layer= [100, 40, 68], input_size= 1 )

our_nn.train(X_train, y_train, lr = 1e-7, epochs = 5000, batch_size= 100, shuffle_indexes= True)

---Before training---
Sum of Ws for layer 0: 86.3854827902921 and sum of bs: 0.0
Sum of Ws for layer 1: 320.7208905230938 and sum of bs: 0.0
Sum of Ws for layer 2: 340.3759188579585 and sum of bs: 0.0
Sum of Ws for layer 3: 7.150046190977206 and sum of bs: 0.0
Epoch: 0
sum of abs dWs for layer 0: 1529706.6705131067 and sum of dbs: 23060.904580599592
sum of Ws for layer 0: 86.38482165490204 and sum of bs: 2.3060904580599593e-05
sum of abs dWs for layer 1: 65531779.29338612 and sum of dbs: 24803.872347957717
sum of Ws for layer 1: 320.7149032996516 and sum of bs: 2.480387234795771e-05
sum of abs dWs for layer 2: 30615673.943302106 and sum of dbs: 50011.07918227741
sum of Ws for layer 2: 340.3740284821264 and sum of bs: 5.001107918227742e-05
sum of abs dWs for layer 3: 12348558.28491178 and sum of dbs: 13972.459256855345
sum of Ws for layer 3: 7.143626382910938 and sum of bs: 1.3972459256855343e-05
MSE on the training set: 25730.541554902866
Epoch: 10
sum of abs dWs for layer 0: 1199403.0

Epoch: 140
sum of abs dWs for layer 0: 363047.92084198253 and sum of dbs: 5473.045699533591
sum of Ws for layer 0: 86.37101293743467 and sum of bs: 0.0008399500489132632
sum of abs dWs for layer 1: 13330616.4154229 and sum of dbs: 5047.476073104294
sum of Ws for layer 1: 320.7018411108446 and sum of bs: 0.001072771945108966
sum of abs dWs for layer 2: 14088349.025117531 and sum of dbs: 26099.819894361917
sum of Ws for layer 2: 340.45611233758734 and sum of bs: 0.004609170340971886
sum of abs dWs for layer 3: 5747546.173608263 and sum of dbs: 7452.490395874758
sum of Ws for layer 3: 7.063858774910409 and sum of bs: 0.0014096481486808741
MSE on the training set: 7374.954585816248
Epoch: 150
sum of abs dWs for layer 0: 375145.2729355916 and sum of dbs: 5655.3525746246405
sum of Ws for layer 0: 86.37259989007578 and sum of bs: 0.0008509198890337044
sum of abs dWs for layer 1: 13433865.820705123 and sum of dbs: 5086.421118101438
sum of Ws for layer 1: 320.7245531493252 and sum of bs: 0.0011

Epoch: 260
sum of abs dWs for layer 0: 367186.93505670084 and sum of dbs: 5535.351063489692
sum of Ws for layer 0: 86.39633120662367 and sum of bs: 0.0011655971829190447
sum of abs dWs for layer 1: 10262719.391755868 and sum of dbs: 3883.326308138581
sum of Ws for layer 1: 321.0165471042659 and sum of bs: 0.0013891059731192309
sum of abs dWs for layer 2: 8231473.34357765 and sum of dbs: 12589.610821636923
sum of Ws for layer 2: 340.6652537744918 and sum of bs: 0.006849581861749065
sum of abs dWs for layer 3: 3198772.9854796794 and sum of dbs: 3408.1323737844064
sum of Ws for layer 3: 7.2757878178111 and sum of bs: 0.0020542740487649947
MSE on the training set: 1529.2049669670728
Epoch: 270
sum of abs dWs for layer 0: 345075.36320148694 and sum of dbs: 5201.036805027304
sum of Ws for layer 0: 86.3984299450714 and sum of bs: 0.0011942315062719487
sum of abs dWs for layer 1: 9797926.293404542 and sum of dbs: 3707.1883165663157
sum of Ws for layer 1: 321.0407908213467 and sum of bs: 0.0014

Epoch: 380
sum of abs dWs for layer 0: 143934.3717111443 and sum of dbs: 2168.156363495225
sum of Ws for layer 0: 86.41496309550149 and sum of bs: 0.0014637048851797937
sum of abs dWs for layer 1: 3886156.1272028573 and sum of dbs: 1468.4797209958415
sum of Ws for layer 1: 321.2331817672713 and sum of bs: 0.0015942908126983014
sum of abs dWs for layer 2: 2158866.8314894047 and sum of dbs: 3005.0675456447393
sum of Ws for layer 2: 340.80538440998225 and sum of bs: 0.007623384474688269
sum of abs dWs for layer 3: 1096819.8933950413 and sum of dbs: 1033.8385569019279
sum of Ws for layer 3: 7.436130296371164 and sum of bs: 0.0022983761086156043
MSE on the training set: 140.18468727692593
Epoch: 390
sum of abs dWs for layer 0: 128328.47578411791 and sum of dbs: 1932.5321754310098
sum of Ws for layer 0: 86.41584525667703 and sum of bs: 0.0014782401866083256
sum of abs dWs for layer 1: 3488858.4952135836 and sum of dbs: 1316.7462018564
sum of Ws for layer 1: 321.2428988175078 and sum of bs: 0

Epoch: 510
sum of abs dWs for layer 0: 32584.00563603035 and sum of dbs: 491.0168719320023
sum of Ws for layer 0: 86.42144043794788 and sum of bs: 0.0015707809689113846
sum of abs dWs for layer 1: 844315.0772322287 and sum of dbs: 319.1591616950945
sum of Ws for layer 1: 321.3045507430844 and sum of bs: 0.001668805267059063
sum of abs dWs for layer 2: 516029.0132095722 and sum of dbs: 698.3886960732718
sum of Ws for layer 2: 340.8504992612479 and sum of bs: 0.007830182474264685
sum of abs dWs for layer 3: 245900.99869092513 and sum of dbs: 222.9052783080905
sum of Ws for layer 3: 7.487017164266524 and sum of bs: 0.002367497114642818
MSE on the training set: 6.500000365469749
Epoch: 520
sum of abs dWs for layer 0: 28769.271553660674 and sum of dbs: 430.0426130243364
sum of Ws for layer 0: 86.42164422917159 and sum of bs: 0.0015741222832828048
sum of abs dWs for layer 1: 764868.149147746 and sum of dbs: 288.89459031234173
sum of Ws for layer 1: 321.30692169417546 and sum of bs: 0.0016711

...

Epoch: 4480
sum of abs dWs for layer 0: 0.00018283181039422243 and sum of dbs: 0.19817891117274192
sum of Ws for layer 0: 86.4232101859278 and sum of bs: 0.0015990831003337785
sum of abs dWs for layer 1: 0.00481047844439827 and sum of dbs: 0.13720547891071694
sum of Ws for layer 1: 321.3249727071858 and sum of bs: 0.0016883880253429587
sum of abs dWs for layer 2: 0.002198180332592034 and sum of dbs: 0.25020678529644225
sum of Ws for layer 2: 340.86268580710043 and sum of bs: 0.007882655579585804
sum of abs dWs for layer 3: 0.0010661275086407368 and sum of dbs: 0.09022077392073853
sum of Ws for layer 3: 7.500588145539435 and sum of bs: 0.0023850212154262244
MSE on the training set: 3.2079213129674535e-06
Epoch: 4490
sum of abs dWs for layer 0: 0.00018283178614626028 and sum of dbs: 0.19817881363196865
sum of Ws for layer 0: 86.423210185929 and sum of bs: 0.0015990815906557763
sum of abs dWs for layer 1: 0.004810478008652466 and sum of dbs: 0.1372054113609445
sum of Ws for layer 1: 321.3

Epoch: 4600
sum of abs dWs for layer 0: 0.00018283071707359534 and sum of dbs: 0.19817774069660146
sum of Ws for layer 0: 86.42321018594218 and sum of bs: 0.0015990649842467985
sum of abs dWs for layer 1: 0.004810451769915906 and sum of dbs: 0.13720466832232658
sum of Ws for layer 1: 321.3249727073578 and sum of bs: 0.001688374491876536
sum of abs dWs for layer 2: 0.0021981730088531416 and sum of dbs: 0.2502053073149189
sum of Ws for layer 2: 340.862685807182 and sum of bs: 0.007882627280865277
sum of abs dWs for layer 3: 0.001066126191381934 and sum of dbs: 0.09022024098962367
sum of Ws for layer 3: 7.500588145579721 and sum of bs: 0.002385010388965596
MSE on the training set: 3.207883415478005e-06
Epoch: 4610
sum of abs dWs for layer 0: 0.0001828305410382917 and sum of dbs: 0.19817764315804026
sum of Ws for layer 0: 86.42321018594336 and sum of bs: 0.0015990634745777135
sum of abs dWs for layer 1: 0.004810447277110169 and sum of dbs: 0.13720460077404997
sum of Ws for layer 1: 321.324

Epoch: 4720
sum of abs dWs for layer 0: 0.00018283003625375015 and sum of dbs: 0.19817657022040014
sum of Ws for layer 0: 86.42321018595653 and sum of bs: 0.0015990468682668237
sum of abs dWs for layer 1: 0.004810436120965986 and sum of dbs: 0.1372038577340482
sum of Ws for layer 1: 321.3249727075299 and sum of bs: 0.001688360958490045
sum of abs dWs for layer 2: 0.0021981714401108814 and sum of dbs: 0.25020382933345237
sum of Ws for layer 2: 340.86268580726363 and sum of bs: 0.007882598982311887
sum of abs dWs for layer 3: 0.001066127958399946 and sum of dbs: 0.09021970805852386
sum of Ws for layer 3: 7.500588145620008 and sum of bs: 0.00238499956256892
MSE on the training set: 3.207845518431698e-06
Epoch: 4730
sum of abs dWs for layer 0: 0.00018282997070165737 and sum of dbs: 0.19817647268070343
sum of Ws for layer 0: 86.42321018595773 and sum of bs: 0.001599045358606656
sum of abs dWs for layer 1: 0.004810434581219921 and sum of dbs: 0.13720379018502302
sum of Ws for layer 1: 321.32

Epoch: 4840
sum of abs dWs for layer 0: 0.00018282890104648065 and sum of dbs: 0.19817539975733361
sum of Ws for layer 0: 86.42321018597087 and sum of bs: 0.0015990287523938546
sum of abs dWs for layer 1: 0.00481040832684118 and sum of dbs: 0.13720304715472872
sum of Ws for layer 1: 321.32497270770193 and sum of bs: 0.0016883474251834863
sum of abs dWs for layer 2: 0.0021981635317615674 and sum of dbs: 0.2502023513684525
sum of Ws for layer 2: 340.8626858073452 and sum of bs: 0.007882570683925633
sum of abs dWs for layer 3: 0.0010661263277540458 and sum of dbs: 0.0902191751333664
sum of Ws for layer 3: 7.500588145660295 and sum of bs: 0.002384988736236195
MSE on the training set: 3.207807621844486e-06
Epoch: 4850
sum of abs dWs for layer 0: 0.00018282879834395256 and sum of dbs: 0.19817530221932497
sum of Ws for layer 0: 86.42321018597207 and sum of bs: 0.0015990272427426036
sum of abs dWs for layer 1: 0.0048104057941058456 and sum of dbs: 0.13720297960684802
sum of Ws for layer 1: 321

Epoch: 4960
sum of abs dWs for layer 0: 0.00018282747861473082 and sum of dbs: 0.19817422930656825
sum of Ws for layer 0: 86.42321018598521 and sum of bs: 0.0015990106366278894
sum of abs dWs for layer 1: 0.004810372855564677 and sum of dbs: 0.13720223658380926
sum of Ws for layer 1: 321.32497270787394 and sum of bs: 0.0016883338919568575
sum of abs dWs for layer 2: 0.0021981516160074896 and sum of dbs: 0.25020087341888264
sum of Ws for layer 2: 340.86268580742683 and sum of bs: 0.007882542385706513
sum of abs dWs for layer 3: 0.0010661225493486508 and sum of dbs: 0.0902186422137769
sum of Ws for layer 3: 7.500588145700583 and sum of bs: 0.0023849779099674204
MSE on the training set: 3.207769725691332e-06
Epoch: 4970
sum of abs dWs for layer 0: 0.0001828273442164494 and sum of dbs: 0.1981741317696921
sum of Ws for layer 0: 86.4232101859864 and sum of bs: 0.0015990091269855555
sum of abs dWs for layer 1: 0.004810369475647765 and sum of dbs: 0.13720216903670052
sum of Ws for layer 1: 321

In [35]:
y_pred, intermediate = our_nn._forward_batch(x0s = X_train)

In [36]:
y_pred

[array([0.00533396]),
 array([2.00542893]),
 array([4.00534492]),
 array([6.0052609]),
 array([8.00517688]),
 array([10.00509287]),
 array([12.00500885]),
 array([14.00492484]),
 array([16.00484082]),
 array([18.00475681]),
 array([20.00467279]),
 array([22.00458877]),
 array([24.00450476]),
 array([26.00442074]),
 array([28.00433673]),
 array([30.00425271]),
 array([32.0041687]),
 array([34.00408468]),
 array([36.00400066]),
 array([38.00391665]),
 array([40.00383263]),
 array([42.00374862]),
 array([44.0036646]),
 array([46.00358059]),
 array([48.00349657]),
 array([50.00341255]),
 array([52.00332854]),
 array([54.00324452]),
 array([56.00316051]),
 array([58.00307649]),
 array([60.00299248]),
 array([62.00290846]),
 array([64.00282444]),
 array([66.00274043]),
 array([68.00265641]),
 array([70.0025724]),
 array([72.00248838]),
 array([74.00240437]),
 array([76.00232035]),
 array([78.00223633]),
 array([80.00215232]),
 array([82.0020683]),
 array([84.00198429]),
 array([86.00190027])

In [ ]:
-7.355385292240312, -7.43845899976959

In [ ]:
y_pred, intermediate = our_nn._forward_batch(x0s = X_train)

In [ ]:
y_pred

In [ ]:
our_unn = NeuralNetwork(number_of_layers= 3, number_of_neurons_per_layer= [200, 33, 128])

In [ ]:
x0s =  [[0 for i in range(58)], [1 for i in range(58)]]

desired_outputs = [0,1]

outputs, intermediate_outputs_list =  our_unn._forward_batch(x0s= x0s )

In [ ]:
len(our_unn.bs)

In [ ]:
output

In [ ]:
dWs, dbiases = our_unn.backward(x0s, outputs, desired_outputs,intermediate_outputs_list )

In [ ]:
len(dWs), len(dbiases)

In [ ]:
# INITIALISE THE PARAMETERS OF YOUR NEURAL NETWORK AND...
# PLEASE CONSIDER TO USE A MASK OF ONE FOR THE ACTION MADE AND ZERO OTHERWISE IF YOU ARE NOT USING VANILLA GRADIENT DESCENT...
# WE SUGGEST A NETWORK WITH ONE HIDDEN LAYER WITH SIZE 200. 


S,X,allowed_a=env.Initialise_game()
N_a=np.shape(allowed_a)[0]   # TOTAL NUMBER OF POSSIBLE ACTIONS

N_in=np.shape(X)[0]    ## INPUT SIZE
N_h=200                ## NUMBER OF HIDDEN NODES


## INITALISE YOUR NEURAL NETWORK...


# HYPERPARAMETERS SUGGESTED (FOR A GRID SIZE OF 4)

epsilon_0 = 0.2     # STARTING VALUE OF EPSILON FOR THE EPSILON-GREEDY POLICY
beta = 0.00005      # THE PARAMETER SETS HOW QUICKLY THE VALUE OF EPSILON IS DECAYING (SEE epsilon_f BELOW)
gamma = 0.85        # THE DISCOUNT FACTOR
eta = 0.0035        # THE LEARNING RATE

N_episodes = 100000 # THE NUMBER OF GAMES TO BE PLAYED 

# SAVING VARIABLES
R_save = np.zeros([N_episodes, 1])
N_moves_save = np.zeros([N_episodes, 1])


In [ ]:
# TRAINING LOOP BONE STRUCTURE...
# I WROTE FOR YOU A RANDOM AGENT (THE RANDOM AGENT WILL BE SLOWER TO GIVE CHECKMATE THAN AN OPTIMISED ONE, 
# SO DON'T GET CONCERNED BY THE TIME IT TAKES), CHANGE WITH YOURS ...

for n in range(N_episodes):

    epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
    Done=0                                   ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
    i = 1                                    ## COUNTER FOR NUMBER OF ACTIONS
    
    S,X,allowed_a=env.Initialise_game()      ## INITIALISE GAME
    print(f'Episode number: {n}')            ## REMOVE THIS OF COURSE, WE USED THIS TO CHECK THAT IT WAS RUNNING.  - Glad to hear.
    
    while Done==0:                           ## START THE EPISODE
        
        
        ## THIS IS A RANDOM AGENT, CHANGE IT...
        
        a,_=np.where(allowed_a==1)
        a_agent=np.random.permutation(a)[0]

                
        S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent)
        
        ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
        if Done==1:
            
            
            break
        
        
        
        
        # IF THE EPISODE IS NOT OVER...
        else:
            
            ## ONLY TO PUT SUMETHING
            PIPPO=1
            
            
        # NEXT STATE AND CO. BECOME ACTUAL STATE...     
        S=np.copy(S_next)
        X=np.copy(X_next)
        allowed_a=np.copy(allowed_a_next)
        
        i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS

        
        
    
    